In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/My Drive/Spring 2025/Directed Study/')
print(os.getcwd()) # This prints the current working directory
print(os.listdir()) # This prints the files in the current working directory

/content/drive/My Drive/Spring 2025/Directed Study
['Directed-Study-Application.pdf', 'SyllabusBobocea.pdf', 'My Directed Study Question Answers.gdoc', 'My Directed Study Question Answers.pdf', 'TM2_project.ipynb', 'Test code', 'Copy of k-TSP.ipynb', 'Shared Directory', 'Final k-TSP.ipynb', 'SST_data']


In [ ]:
# imports
import pandas as pd
import numpy as np
import matplotlib as plt
from copy import deepcopy
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Implementation of K-TSP Classifier

### Intuitive Explanation:
The basic idea is to find pairs of features that give the most information. This is done by scoring the features whose relative difference changes the most from one class to another. Thus their difference can indicate what class we should predict




Though the algorithm's pseudocode is laid out in the figure, below are some extra details that might not be explicitly stated in the paper that guided my implementation. The first and maybe most important step was calculating all the matrices that the algorithm required.
### Calculate all necessary matrices:
* The first matrix we need is the rank matrix $R \in \mathbb{R}^
{PxN}$. Our data $X \in \mathbb{R}^{PxN}$ has the same shape as well. The rank assigned to each entry in $X$ is simply calculated by assigning the highest rank to the highest value in that profile (profiles are columns here).
* For our score matrix $\Delta$ we have the formula $\Delta_{ij} = |p_{ij}(C_1) - p_{ij}(C_2)|$, where $C_m$ are the possible classes for $m \in {1,2}$, and $p_{ij}(C_m) = Prob(R_i < R_j | Y = C_m)$.
  * This probability is calculated by the relative frequencies of the ranks of certain genes $i$ and $j$ within profiles of a given label, that is:
  $p_{ij}(C_m) = \frac{\sum_{k=1}^N \mathbb{1}(y_k = C_m) \cdot \mathbb{1}(R_{i,k} < R_{j,k})}{\sum_{k=1}^N \mathbb{1}(y_k = C_m)}$, where $\mathbb{1}(\cdot)$ is the indicator function. In keeping with the notation of the paper, this is alternatively expressed as, $p_{ij}(C_m) = \frac{\sum_{n \in C_m} \mathbb{1}(R_{i,n} < R_{j,n})}{|C_m|}$ where $|C_m|$ is the number of samples in $C_m$.
* Lastly we have the "rank score" matrix $\Gamma$ that is used to break ties. Each entry is calculates as follows, $\Gamma_{ij} = |\gamma_{ij}(C_1) = \gamma_{ij}(C_2)|$ where $\gamma_{ij}  = \frac{\sum_{n \in C_m} (R_{i,n} - R_{j,n})}{|C_m|}$.

These were the explicit formulas I used to be able to implement the algorithm, the rest of the algorithm is written clearly in the pseudocode of the paper.




In [ ]:
# Algorithm for k-TSP
class KTSP:

  def __init__(self):
    self.data = None
    self.labels = None
    self.rank_matrix = None
    self.best_pair = None
    self.top_k_pairs = None

  def compute_Rank(self, reduced_data : pd.DataFrame):
    """
    for i in range(reduced_data.shape[1]):
      sorted_col = reduced_data[i].sort_values()
      rank = 1
      for num in sorted_col:
        reduced_data[i].loc[num] = rank
        rank += 1
    """
    # faster way based on built-in pandas method
    self.rank_matrix = reduced_data.rank(method="first", ascending=False, axis=0)  # Compute ranks per column
    return self.rank_matrix.astype(int)


  def compute_Delta(self):
    # initializing the delta matrix, it's P X P symmetric
    delta =  np.zeros((self.P,self.P))

    # we subset the matrix based on classes
    class_0 = self.rank_matrix.loc[:, self.labels == 0]
    class_1 = self.rank_matrix.loc[:, self.labels == 1]

    for i in range(self.P):
      for j in range(i+1,self.P):
        p_class0 = (class_0.iloc[i] < class_0.iloc[j]).mean()
        p_class1 = (class_1.iloc[i] < class_1.iloc[j]).mean()
        delta[i, j] = abs(p_class0 - p_class1)
        delta[j, i] = delta[i, j]

    return pd.DataFrame(delta)

  def compute_Gamma(self):
    # initializing the gamma matrix which is also P X P
    gamma_mat = np.zeros((self.P,self.P))

    # check that this gets all columns with the corresponding label
    class_0 = self.rank_matrix.loc[:, self.labels == 0].to_numpy()
    class_1 = self.rank_matrix.loc[:, self.labels == 1].to_numpy()

    for i in range(self.P):
      for j in range(i+1, self.P):
        #check this
        gamma_0 = np.mean(class_0[i, :] - class_0[j, :])
        gamma_1 = np.mean(class_1[i, :] - class_1[j, :])
        gamma_mat[i, j] = abs(gamma_0 - gamma_1)
        gamma_mat[j, i] = gamma_mat[i, j]

    return pd.DataFrame(gamma_mat)


  def get_ordered_list(self, delta_mat, gamma_mat):
    # stack Delta and Gamma to sort pairs by Delta, then Gamma
    delta_stacked = delta_mat.stack()
    gamma_stacked = gamma_mat.stack()
    pairs_df = pd.DataFrame({
        'delta': delta_stacked,
        'gamma': gamma_stacked
    })
    # sort by Delta, then Gamma
    pairs_df = pairs_df.sort_values(by=['delta', 'gamma'], ascending=[False, False])
    # select top k pairs
    top_pairs = [(i, j) for i, j in pairs_df.index if i < j]
    return top_pairs


  def remove_pair(self, top_pair, list_O):
    return [(i,j) for (i,j) in list_O if i not in top_pair and j not in top_pair]


  def cross_val_k(self):
    k_upper_bound = 10 # should be even, since it'll exclude the last number in the loop below
    N = self.labels.shape[0]
    little_n = 3
    m = N // little_n
    sample_indices = np.arange(self.data.shape[1])
    np.random.shuffle(sample_indices)
    error_rates = {}

    for fold in range(m):
      test_indices = sample_indices[fold*little_n : (fold+1)*little_n]
      train_indices = np.setdiff1d(sample_indices, test_indices)

      X_train = self.data.iloc[:, train_indices]
      X_test = self.data.iloc[:, test_indices]
      y_train = self.true_labels.iloc[train_indices]
      y_test = self.true_labels.iloc[test_indices]

      self.labels = pd.Series(y_train.values, index=X_train.columns)
      self.P = X_train.shape[0]
      self.compute_Rank(X_train)
      delta_mat = self.compute_Delta()
      gamma_mat = self.compute_Gamma()

      list_O = self.get_ordered_list(delta_mat, gamma_mat)
      list_Theta = []
      for k in range(1,k_upper_bound):
        list_Theta.append(list_O[0])
        list_O = self.remove_pair(list_O[0], list_O)
        if k % 2 != 0:
          self.top_k_pairs = list_Theta
          self.optimal_k = k
          if self.optimal_k == 1:
            self.best_pair = self.top_k_pairs[0]
          predictions = self.predict(X_test)
          accuracy = (predictions == y_test.values).mean()
          # print(f"Accuracy for k={k} is: {accuracy}")
          error_rate = 1 - accuracy
          error_rates[k] = error_rate

    self.optimal_k = min(error_rates, key=error_rates.get)
    self.labels = pd.Series(self.true_labels.values, index=self.data.columns)
    self.P = self.data.shape[0]
    self.compute_Rank(self.data)
    delta_mat = self.compute_Delta()
    gamma_mat = self.compute_Gamma()
    ordered_list = self.get_ordered_list(delta_mat, gamma_mat)
    self.top_k_pairs = []
    for _ in range(self.optimal_k):
      self.top_k_pairs.append(ordered_list[0])
      ordered_list = self.remove_pair(ordered_list[0], ordered_list)
    return self.optimal_k

  def fit(self, data, labels, k_cross_val=True, k=None, verbose=True):
    self.true_labels = deepcopy(labels)
    self.k_cross_val = k_cross_val
    if (self.k_cross_val == False and (k == None or k == 1)) or k == 1:
      self.optimal_k = 1
      self.data = data
      self.labels = labels
      # this makes sure that we have the correct orientation for the data
      if len(self.labels) == self.data.shape[0]:
          self.data = self.data.T
      elif len(self.labels) != self.data.shape[1]:
          raise Exception(f"Number of samples in data ({self.data.shape[1]}) must match length of labels ({len(self.labels)})")

      self.labels = pd.Series(self.labels.values, index=self.data.columns)
      self.P = self.data.shape[0]
      self.compute_Rank(self.data)
      delta_mat = self.compute_Delta()
      gamma_mat = self.compute_Gamma()

      delta_arr = delta_mat.to_numpy()
      max_delta = delta_arr.max()
      max_indices = np.where(delta_arr == max_delta)
      max_pairs = list(zip(max_indices[0], max_indices[1]))

      if len(max_pairs) == 1:
        return max_pairs[0]
      else:
        max_gamma = -float('inf')
        self.best_pair = None
        for i, j in max_pairs:
            gamma_value = gamma_mat.iloc[i, j]
            if gamma_value > max_gamma:
                max_gamma = gamma_value
                self.best_pair = (i, j)
        return self.best_pair
    else:
      self.data = data
      self.labels = labels
      if k == None:
        self.cross_val_k()
        if verbose:
          print(f"Optimal k: {self.optimal_k}")
          print("Best pair:", self.top_k_pairs)
          print("Best pair:", self.best_pair)
      else:
        self.optimal_k = k
        self.labels = pd.Series(self.true_labels.values, index=self.data.columns)
        self.P = self.data.shape[0]
        self.compute_Rank(self.data)
        delta_mat = self.compute_Delta()
        gamma_mat = self.compute_Gamma()
        ordered_list = self.get_ordered_list(delta_mat, gamma_mat)
        self.top_k_pairs = []
        for _ in range(self.optimal_k):
          self.top_k_pairs.append(ordered_list[0])
          ordered_list = self.remove_pair(ordered_list[0], ordered_list)



  def predict(self, new_samples):
    if isinstance(new_samples, pd.Series):
        new_samples = new_samples.to_frame().T
    if (self.k_cross_val == False and (self.optimal_k == None or self.optimal_k == 1)) or self.optimal_k == 1:
      i = self.best_pair[0]
      j = self.best_pair[1]

      # compute p_ij(C_m)
      class_0 = self.rank_matrix.loc[:, self.labels == 0]
      class_1 = self.rank_matrix.loc[:, self.labels == 1]
      p_class0 = (class_0.iloc[i] < class_0.iloc[j]).mean()
      p_class1 = (class_1.iloc[i] < class_1.iloc[j]).mean()

      # decision rule
      predictions = []
      for _, sample in new_samples.items():
        new_sample = pd.Series(sample, index=self.data.index)
        new_ranks = pd.Series(new_sample).rank(method="first", ascending=False)
        votes = {0: 0, 1: 0}
        if p_class0 > p_class1:
          if new_ranks.iloc[i] < new_ranks.iloc[j]:
            votes[0] += 1
          else:
            votes[1] += 1
        else:
          if new_ranks.iloc[i] < new_ranks.iloc[j]:
            votes[1] += 1
          else:
            votes[0] += 1
        predictions.append(0 if votes[0] > votes[1] else 1)
      return np.array(predictions)
    else:
      if self.top_k_pairs is None:
        raise Exception("Must call fit() to select top k pairs before making predictions")

      predictions = []
      for _, sample in new_samples.items():
        new_sample = pd.Series(sample, index=self.data.index)
        new_ranks = pd.Series(new_sample).rank(method="first", ascending=False)
        votes = {0: 0, 1: 0}
        for i, j in self.top_k_pairs:
          class_0 = self.rank_matrix.loc[:, self.labels == 0]
          class_1 = self.rank_matrix.loc[:, self.labels == 1]
          p_class0 = (class_0.iloc[i] < class_0.iloc[j]).mean()
          p_class1 = (class_1.iloc[i] < class_1.iloc[j]).mean()
          if p_class0 > p_class1:
            if new_ranks.iloc[i] < new_ranks.iloc[j]:
                votes[0] += 1
            else:
                votes[1] += 1
          else:
            if new_ranks.iloc[i] < new_ranks.iloc[j]:
                votes[1] += 1
            else:
                votes[0] += 1
        predictions.append(0 if votes[0] > votes[1] else 1)
      return np.array(predictions)

In [ ]:
def pre_process(filename, n_features):
  pros = pd.read_csv(filename, delimiter=',', header=None, low_memory=False)
  X = pros.iloc[1:]
  X = X.apply(pd.to_numeric, errors='coerce')
  y = pros.iloc[0]
  y = y.replace({'Tumor': 1, 'Normal': 0})
  print(f"Original data shape for X: {X.shape} and y: {y.shape}")

  # The code below is only if you want to limit the amount of features used

  num_features_to_keep = n_features
  print(f"Keeping the top {num_features_to_keep} features with the highest variance")
  variances = X.var(axis=1)
  top_features = variances.nlargest(num_features_to_keep).index
  X = X.loc[top_features]
  y = y.iloc[:X.shape[1]]

  return X, y

In [ ]:
def jitter(X, y):
  # first I have to identify the minority and majority class of the dataset
  # add code to get counts of both classes
  y_list = list(y)
  counts = y.value_counts().to_dict()
  #print("Class counts:", y.value_counts())
  class_0_count = counts[0]
  class_1_count = counts[1]
  if class_0_count == class_1_count:
    return X, y

  # for the class that is the minority
  # write code that takes data points at random and adds gaussian noise
  # leave the variance visible so as to be tweaked
  if class_0_count < class_1_count:
    diff = class_1_count - class_0_count
    class_0_indices = np.where(y == 0)[0]


    # the std dev we use is one tenth of the std dev of the feature within that class
    # IMPORTANT: SHOULD CHANGE THIS TO A FRACTION OF THE DISTANCE TO THE NEAREST NEIGHBOR, VARIANCES ARE TOO LARGE FOR FEATURES
    std_dev = np.std(X.iloc[:, class_0_indices].to_numpy(), axis=1, keepdims=True)
    alpha = 0.05


    indices = np.random.choice(class_0_indices, size=diff, replace=True)
    selected_samples = X.iloc[:, indices]
    noise = np.random.normal(0.0, alpha * std_dev, selected_samples.shape)
    jittered_samples = selected_samples + noise
    y_list.extend([0] * diff)

    # print(f"Example of a jittered sample:\n")
    # print(f"Original: {selected_samples.iloc[:5, 1]} \n")
    # print(f"Jittered: {jittered_samples.iloc[:5, 1]}")

  elif class_1_count < class_0_count:
    diff = class_0_count - class_1_count
    class_1_indices = np.where(y == 1)[0]
    std_dev = np.std(X.iloc[:, class_0_indices].to_numpy(), axis=1, keepdims=True)
    alpha = 0.05


    indices = np.random.choice(class_1_indices, size=diff, replace=True)
    selected_samples = X.iloc[:, indices]
    noise = np.random.normal(0, alpha * std_dev, selected_samples.shape)
    jittered_samples = selected_samples + noise
    y_list.extend([1] * diff)

    # print(f"Example of a jittered sample:\n")
    # print(f"Original: \n{selected_samples.iloc[:, 1]} \n")
    # print(f"Jittered: \n{jittered_samples.iloc[:, 1]}")


  X_aug = pd.concat([X, jittered_samples], axis=1)
  y_aug = pd.Series(y_list)
  X_aug.columns = range(0, X_aug.shape[1])
  y_aug.index = range(0, y_aug.shape[0])
  print(f"New data shape for X: {X_aug.shape}, and y: {y_aug.shape}")
  return X_aug, y_aug



In [ ]:
def data_split(X, y):
  # can modify test_size and var
  X, y = jitter(X, y)
  return train_test_split(X.T, y, test_size=0.3)

In [ ]:
def train_and_report(X_train, X_test, y_train, y_test):
  X_train = X_train.T
  X_test = X_test.T
  tsp = KTSP()
  best_pair = tsp.fit(X_train, y_train, k_cross_val=True, verbose=True)
  y_pred = tsp.predict(X_test)
  return classification_report(y_test, y_pred, output_dict=True)

In [ ]:
def cross_val(X, y, k=3):
  for i in range(k):
    print(f"Fold: {i+1}")
    X_train, X_test, y_train, y_test = train_test_split(X.T, y, test_size=0.2)
    report = train_and_report(X_train, X_test, y_train, y_test)
    print(f"Fold {i+1} Report:\n {report}")
    if i == 0:
      averages = deepcopy(report)
    else:
      for key in report:
        if key == 'accuracy':
          averages['accuracy'] += report['accuracy']
          continue
        if key == 'recall':
          averages['recall'] += report['recall']
          continue
        for metric in report[key]:
            averages[key][metric] += report[key][metric]


  for key in averages:
    if key == 'accuracy':
          averages['accuracy'] = averages['accuracy'] / k
          continue
    for metric in averages[key]:
         averages[key][metric] = averages[key][metric] / k

  return averages


In [ ]:
def format_classification_report(report):
    lines = []
    headers = ["precision", "recall", "f1-score", "support"]
    line_fmt = "{:>12} {:>10} {:>10} {:>10} {:>10}"
    lines.append(line_fmt.format("", *headers))
    for label in report:
        if isinstance(report[label], dict):
            values = [report[label].get(h, 0) for h in headers]
            lines.append(line_fmt.format(label, *["{:.4f}".format(v) if isinstance(v, float) else str(v) for v in values]))
        elif label == "accuracy":
            lines.append("{:>12} {:>10.4f}".format("accuracy", report[label]))
    return "\n".join(lines)

In [ ]:
def run_pipeline(filename):
  n_features = 100
  X, y = pre_process(filename, n_features)
  report = cross_val(X, y)
  print(format_classification_report(report))

In [ ]:
X_train.iloc[21]

,Sep_atlantic
0,301.87653
1,301.16852
2,301.13016
3,301.82733
4,301.82360
...,...
85,301.27988
86,301.24844
87,301.29430
88,301.12796


In [ ]:
df = pd.read_csv("SST_data")
X = df.iloc[:, 1:-2]
y_train = df.iloc[:, -1] # Keep y as a pandas Series

X_train = X.T
tsp = KTSP()
best_pair = tsp.fit(X_train, y_train, k_cross_val=True, verbose=True)


Optimal k: 5
Best pair: [(37, 39), (36, 38), (7, 24), (8, 26), (21, 22)]
Best pair: (37, 38)


In [ ]:
df = pd.read_csv("SST_data")
X = df.iloc[:, 1:-2]
y = df.iloc[:, -1] # Keep y as a pandas Series

X = X.T
report = cross_val(X, y)
print(format_classification_report(report))

Fold: 1
Optimal k: 1
Fold 1 Report:
 {'0': {'precision': 0.8333333333333334, 'recall': 0.625, 'f1-score': 0.7142857142857143, 'support': 8.0}, '1': {'precision': 0.75, 'recall': 0.9, 'f1-score': 0.8181818181818182, 'support': 10.0}, 'accuracy': 0.7777777777777778, 'macro avg': {'precision': 0.7916666666666667, 'recall': 0.7625, 'f1-score': 0.7662337662337663, 'support': 18.0}, 'weighted avg': {'precision': 0.7870370370370371, 'recall': 0.7777777777777778, 'f1-score': 0.7720057720057719, 'support': 18.0}}
Fold: 2
Optimal k: 1
Fold 2 Report:
 {'0': {'precision': 0.5, 'recall': 0.16666666666666666, 'f1-score': 0.25, 'support': 6.0}, '1': {'precision': 0.6875, 'recall': 0.9166666666666666, 'f1-score': 0.7857142857142857, 'support': 12.0}, 'accuracy': 0.6666666666666666, 'macro avg': {'precision': 0.59375, 'recall': 0.5416666666666666, 'f1-score': 0.5178571428571428, 'support': 18.0}, 'weighted avg': {'precision': 0.625, 'recall': 0.6666666666666666, 'f1-score': 0.6071428571428572, 'support

In [ ]:
run_pipeline("Prostate1.txt")

<ipython-input-71-130fb631c538>:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y = y.replace({'Tumor': 1, 'Normal': 0})


Original data shape for X: (12600, 102) and y: (102,)
Keeping the top 100 features with the highest variance
Fold: 1
New data shape for X: (100, 104), and y: (104,)
Optimal k: 9
Fold 1 Report:
 {'0': {'precision': 0.7857142857142857, 'recall': 0.8461538461538461, 'f1-score': 0.8148148148148148, 'support': 13.0}, '1': {'precision': 0.8888888888888888, 'recall': 0.8421052631578947, 'f1-score': 0.8648648648648649, 'support': 19.0}, 'accuracy': 0.84375, 'macro avg': {'precision': 0.8373015873015872, 'recall': 0.8441295546558705, 'f1-score': 0.8398398398398399, 'support': 32.0}, 'weighted avg': {'precision': 0.8469742063492063, 'recall': 0.84375, 'f1-score': 0.8445320320320321, 'support': 32.0}}
              precision     recall   f1-score    support
           0     0.7857     0.8462     0.8148    13.0000
           1     0.8889     0.8421     0.8649    19.0000
    accuracy     0.8438
   macro avg     0.8373     0.8441     0.8398    32.0000
weighted avg     0.8470     0.8438     0.8445   

In [ ]:
run_pipeline("Colon.txt")

<ipython-input-80-87a923ab50a1>:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y = y.replace({'Tumor': 1, 'Normal': 0})


Original data shape for X: (2000, 62) and y: (62,)
Keeping the top 100 features with the highest variance
Fold: 1
New data shape for X: (100, 80), and y: (80,)
Optimal k: 1
Fold 1 Report:
 {'0': {'precision': 0.625, 'recall': 0.9090909090909091, 'f1-score': 0.7407407407407407, 'support': 11.0}, '1': {'precision': 0.875, 'recall': 0.5384615384615384, 'f1-score': 0.6666666666666666, 'support': 13.0}, 'accuracy': 0.7083333333333334, 'macro avg': {'precision': 0.75, 'recall': 0.7237762237762237, 'f1-score': 0.7037037037037037, 'support': 24.0}, 'weighted avg': {'precision': 0.7604166666666666, 'recall': 0.7083333333333334, 'f1-score': 0.7006172839506172, 'support': 24.0}}
Fold: 2
New data shape for X: (100, 80), and y: (80,)
Optimal k: 1
Fold 2 Report:
 {'0': {'precision': 0.8, 'recall': 0.8, 'f1-score': 0.8, 'support': 10.0}, '1': {'precision': 0.8571428571428571, 'recall': 0.8571428571428571, 'f1-score': 0.8571428571428571, 'support': 14.0}, 'accuracy': 0.8333333333333334, 'macro avg': {

Here is a [link](https://www.nb-data.com/p/breaking-down-the-classification) that details the breakdown/meaning of everything in the classification report

#### Structure of the pipeline
1. Start with a function that loads/pre-processes the data

    a. Input: filename; Output: X, y datasets

2. Then have function that takes X, y and creates stratified test and train sets (will eventually implement copy then jitter for creating extra data)

    a. Input: X, y; Output: X_train, y_train, X_test, y_test

3. Then I will have a function that trains on the train data, then does prediction on the test data, this will report a number of different metrics for the accuracy of the classifier

    a. Input: X_train, y_train, X_test, y_test; Output: Accuracy Report

4. Can have one more function that repeats steps 2 and 3 with different sets for cross-validation and then (I think) I average the reports
